**Trabajo MOVIX G16**  
*Inteligencia Artificial I*


Inteligencia Artificial I
Práctica 1.
(Sesión 3) Búsqueda heurística
El objetivo del puzle MovIX es utilizar el mínimo número de movimientos para
alinear (horizontal, vertical o diagonal) en un tablero NxN un conjunto X (X<= N) de
fichas con diferentes habilidades de movimiento.
Las celdas del tablero inicial pueden estar vacías o contener una ficha. El jugador
puede mover una ficha en cada turno y dos fichas no pueden ocupar la misma
casilla al mismo tiempo.


Hay varios tipos de fichas que tienen distintas habilidades de movimiento:

S- Saltadora: puede saltar sobre una ficha adyacente (horizontal o vertical)
y aterrizar en la siguiente celda vacía. No puede moverse si no es saltando
sobre una ficha, es decir, no se puede saltar sobre celdas vacías, ni sobre
dos fichas, ni sobre un muro (si se incluyen).


L- Lenta: puede moverse únicamente a una celda adyacente vacía en
cualquier dirección (horizontal, vertical o diagonal).


V- Vertical: puede moverse verticalmente a cualquier celda vacía en la
misma columna sin saltar fichas.


H- Horizontal: puede moverse horizontalmente a cualquier celda vacía en
la misma fila sin saltar fichas. Ejemplo: Si está en (1, 2) y toda la fila 1 está
vacía podría moverse a (1, 1), (1, 3), (1, 4), (1, 0).


La siguiente configuración es un ejemplo de la posición inicial de las fichas (al
inicio se coloca aleatoriamente un número dado (Y) de fichas de cada tipo. En el
ejemplo Y=1:
Se pide:


o Representa el problema en AIMA para resolverlo con búsqueda
heurística. Analiza el tamaño y forma del espacio de estados.


o Define varias H’ para el problema y analízalas comparando su
rendimiento al resolver distintos estados iniciales y distinto número
de fichas iniciales.

o Realiza pruebas con distintos tamaños de tablero N, de línea X, y
número de fichas iniciales de cada tipo (Y).


o Elige de forma justificada cual es la mejor h’ para este problema
o Compara la búsqueda heurística con algún algoritmo de búsqueda
ciega para medir las ventajas del uso de la heurística.


o Añade un nuevo tipo de casillas M muro sobre las que no se puede
saltar, es decir, un muro detiene el movimiento de las fichas V
vertical, H horizontal. Prueba el efecto que tiene en la resolución de
tableros usando las mismas heurísticas.


Ideas para definir heurísticas:


 Ten en cuenta si hay fichas ya alineadas y cuántas fichas adicionales se
necesitarían para completar la alineación de X fichas.


 Considera los conflictos entre fichas que impiden la alineación. Cada ficha
que está en la línea de alineación pero que no puede moverse (por ser de
un tipo que no permite el movimiento adecuado) suma un costo a la
heurística. Cuenta cuántas fichas están en la misma línea que no pueden
alinearse debido a sus restricciones de movimiento. Cada conflicto podría
sumar un valor (por ejemplo, 1 por cada conflicto).


 Combina las heurísticas anteriores para dar un valor más integral. Podrías
ponderar la distancia de Manhattan y la alineación parcial para crear una
heurística más completa. 

In [6]:

import math
from search import Problem, breadth_first_tree_search,astar_search

class MovIX(Problem):

    def __init__(self, initial, numpieces, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        
        initial_data = []
      
        self.InicializarEstadoInicial(initial, initial_data)
        
        self.initial = tuple(initial_data)  # Convierte a tupla de tuplas
        
        self.X = numpieces 
        self.N = len(initial)  
        self.goal = goal

    def actions(self, state):
        N = self.N
        possible_actions = []

        for i, (tipo, posicion) in enumerate(state):  # Cada pieza es una tupla (tipo, posición)
            x, y = posicion

            if tipo == 'S':  # Ficha Saltadora
                adyacentes = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
                for adj_x, adj_y in adyacentes:
                    if 0 <= adj_x < N and 0 <= adj_y < N:
                        salto_x = adj_x + (adj_x - x)
                        salto_y = adj_y + (adj_y - y)
                        if (0 <= salto_x < N and 0 <= salto_y < N and 
                            self.is_empty(state, (salto_x, salto_y)) and
                            not self.is_empty(state, (adj_x, adj_y))):
                            possible_actions.append((i, (salto_x, salto_y)))

            elif tipo == 'L':  # Ficha Lenta
                movimientos = [(x-1, y), (x+1, y), (x, y-1), (x, y+1), (x-1, y-1), (x-1, y+1), (x+1, y-1), (x+1, y+1)]
                for mov_x, mov_y in movimientos:
                    if 0 <= mov_x < N and 0 <= mov_y < N and self.is_empty(state, (mov_x, mov_y)):
                        possible_actions.append((i, (mov_x, mov_y)))

            elif tipo == 'V':  # Ficha Vertical
                for mov_x in range(N):
                    if mov_x != x and self.is_empty(state, (mov_x, y)):
                        possible_actions.append((i, (mov_x, y)))

            elif tipo == 'H':  # Ficha Horizontal
                for mov_y in range(N):
                    if mov_y != y and self.is_empty(state, (x, mov_y)):
                        possible_actions.append((i, (x, mov_y)))

        print(possible_actions)
        return possible_actions

    def result(self, state, action):
        index, new_position = action
        new_state = list(state)  # Convertimos la tupla original en una lista temporal
        pieza, _ = new_state[index]
        new_state[index] = (pieza, new_position)
        return tuple(new_state)  # Convertimos de nuevo en tupla de tuplas




    def goal_test(self, state) -> bool:
        if self.X <= 1:
            return True  # If only one point, alignment is trivial

        # Extract positions of pieces
        posiciones = [pos for _, pos in state]

        # Set up first two pieces to calculate initial slope
        firstPiece, secondPiece = posiciones[0], posiciones[1]
        firsti, firstj = firstPiece
        secondi, secondj = secondPiece

        # Calculate deltas
        deltai, deltaj = secondi - firsti, secondj - firstj

        # Determine initial slope, handling division by zero
        if deltaj == 0:
            currentSlope = float('inf')  # Vertical line
        else:
            currentSlope = deltai / deltaj  # Calculate slope normally

        # Check alignment constraints based on slope
        if abs(deltai) >= self.X or abs(deltaj) >= self.X:
            return False  # Gaps exist in the line
        if abs(currentSlope) not in {0, 1} and not math.isinf(currentSlope):
            return False  # Slope must be 0, ±1, or infinity

        # Check that all pieces follow the initial slope
        for i in range(2, self.X):
            secondi, secondj = posiciones[i]
            deltai, deltaj = secondi - firsti, secondj - firstj

            # Calculate slope with handling for vertical lines
            if deltaj == 0:
                slope = float('inf')
            else:
                slope = deltai / deltaj

            # Ensure slope consistency
            if slope != currentSlope:
                return False
            # Check for gaps
            if abs(deltai) >= self.X or abs(deltaj) >= self.X:
                return False

        return True

           
    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError
    
    def mostrar_solucion(problem, node):
        estados = [problem.initial]
        acciones = node.solution()

        print("Pasos desde el estado inicial hasta el objetivo:\n")
        
        for i, action in enumerate(acciones):
            nuevo_estado = problem.result(estados[-1], action)
            
            # Representación del tablero en texto
            N = problem.N
            tablero = [['_' for _ in range(N)] for _ in range(N)]
            for tipo, (x, y) in nuevo_estado:
                tablero[x][y] = tipo
            
            # Mostrar el tablero y la acción
            print(f"Paso {i+1}: Acción - {action}")
            for row in tablero:
                print(" ".join(row))
            print("\n")
            
            estados.append(nuevo_estado)
        
        print("Solución completa.")

    
    def InicializarEstadoInicial(self, tablero, data):
        for i, row in enumerate(tablero):
            for j, cell in enumerate(row):
                if cell != "_":  # If there's a piece in this cell
                    data.append((cell, (i, j)))  # Append tuple with piece type and position
    

    # Método auxiliar para ver si una celda está vacía
    def is_empty(self, state, position):
        return position not in [pos for _, pos in state]

    #Por defecto si la heurística es nula acude a esta
    def h(self, node):
        
        return 0
    

In [7]:
tableroEjemplo = [
    ['_', '_', '_', '_', '_'],
    ['_', 'S', '_', 'H', '_'],
    ['_', '_', 'L', '_', '_'],
    ['V', '_', '_', '_', '_'],
    ['_', '_', '_', '_', '_']
]


In [8]:

# estado inicial

initial_board = [
    ['S', '_', '_', '_', '_'],
    ['V', '_', '_', '_', '_'],
    ['L', '_', '_', '_', '_'],
    ['_', 'H', '_', '_', '_'],
    ['_', '_', '_', '_', '_']
]

X = 4

def partial_alignment_heuristic(node):
    """
    Calcula la heurística basada en la alineación parcial.
    Cuenta cuántas fichas están alineadas y cuántas más se necesitan para completar la alineación.
    """
    max_alignment_count = 1  # Siempre hay al menos alguna ficha
    
    rows = {}
    cols = {}
    up_diagonals = {}  # Diagonales con pendientes 1 y -1
    down_diagonals = {}  # Diagonales con pendientes 1 y -1        

    for tipo, location in node.state:
        x, y = location
        d1 = x - y
        d2 = x + y
        rows[x] = rows.get(x, 0) + 1
        cols[y] = cols.get(y, 0) + 1
        down_diagonals[d1] = down_diagonals.get(d1, 0) + 1
        up_diagonals[d2] = up_diagonals.get(d2, 0) + 1
        
        max_alignment_count = max(rows[x], max_alignment_count)
        max_alignment_count = max(cols[y], max_alignment_count)
        max_alignment_count = max(down_diagonals[d1], max_alignment_count)
        max_alignment_count = max(up_diagonals[d2], max_alignment_count)
        
        if max_alignment_count == X:
            break  # mejor alineación no se va a encontrar
    
    return X - max_alignment_count


def manhattan_heuristic(node):
        #tamaño del tablero (NxN)
        N = len(node.state)
        print("N es: ", len(node.state))
        #creo un punto de alineación de referencia en el centro del tablero, y para calcularlo divido entre 2 las coordenadas X e Y
        alineada_x, alineada_y = N //2, N //2 #división entera

        print("entrooo\n")
        
        #distancia total al la linea de meta, en este caso el centro del tablero
        distancia_total = 0

        #se rrecorren todas las posiciones de las fichas en el estado actual
        #print("node.state.state es:  ", node.state.state)
        print("node.state es:  ", node.state)
        
        for pieza, posicion in node.state: #hago el for con elemento (que es la tupla de tipo de ficha y posicion) porque si meto una tupla directamente peta el kernel
            #calclulo la distancia desde su posicion incial hasta el punto de alineación en el centro del tablero
            x, y = posicion
            distancia = abs(x - alineada_x) + abs(y - alineada_y)
            print(distancia)
            distancia_total += distancia #sumo la distancia a la distancia total"""

        return distancia_total #devuelvo la distancia total como valor heurístico


In [9]:
print("*************************************************************BUSQUEDA NO INFORMADA (EN ANCHURA)*****************************************************************************\n")

problema = MovIX(tableroEjemplo, numpieces=X)
# búsqueda DFS
solution = breadth_first_tree_search(problema)
# mostrar solución
if solution:
    MovIX.mostrar_solucion(problema, solution)
else:
    print("No se encontró solución.")


print("*************************************************************BUSQUEDA HEURÍSTICA*****************************************************************************\n")

print("********************************************HEURISTICA DE ALINEAMIENTO PARCIAL*******************************************************")
problema = MovIX(tableroEjemplo, numpieces=X)

solution = astar_search(problema,partial_alignment_heuristic)

if solution:
    MovIX.mostrar_solucion(problema, solution)
else:
    print("No se encontró solución.")

*************************************************************BUSQUEDA NO INFORMADA (EN ANCHURA)*****************************************************************************

[(1, (1, 0)), (1, (1, 2)), (1, (1, 4)), (2, (1, 2)), (2, (3, 2)), (2, (2, 1)), (2, (2, 3)), (2, (3, 1)), (2, (3, 3)), (3, (0, 0)), (3, (1, 0)), (3, (2, 0)), (3, (4, 0))]
[(1, (1, 2)), (1, (1, 3)), (1, (1, 4)), (2, (1, 2)), (2, (3, 2)), (2, (2, 1)), (2, (2, 3)), (2, (1, 3)), (2, (3, 1)), (2, (3, 3)), (3, (0, 0)), (3, (2, 0)), (3, (4, 0))]
[(0, (1, 3)), (1, (1, 0)), (1, (1, 3)), (1, (1, 4)), (2, (3, 2)), (2, (2, 1)), (2, (2, 3)), (2, (1, 3)), (2, (3, 1)), (2, (3, 3)), (3, (0, 0)), (3, (1, 0)), (3, (2, 0)), (3, (4, 0))]
[(1, (1, 0)), (1, (1, 2)), (1, (1, 3)), (2, (1, 2)), (2, (3, 2)), (2, (2, 1)), (2, (2, 3)), (2, (1, 3)), (2, (3, 1)), (2, (3, 3)), (3, (0, 0)), (3, (1, 0)), (3, (2, 0)), (3, (4, 0))]
[(1, (1, 0)), (1, (1, 4)), (2, (0, 2)), (2, (2, 2)), (2, (0, 1)), (2, (0, 3)), (2, (2, 1)), (2, (2, 3)), (3, (0, 0)), (3

In [10]:
print("********************************************HEURISTICA MANHATTAN*******************************************************")

problema = MovIX(tableroEjemplo, numpieces=X)

solution = astar_search(problema,manhattan_heuristic, True)

if solution:
    MovIX.mostrar_solucion(problema, solution)
else:
    print("No se encontró solución.")


problema = MovIX(tableroEjemplo, numpieces=X)

# búsqueda DFS
%timeit sum(range(1000))
 #contar nodos
solution = breadth_first_tree_search(problema)

# mostrar solución
if solution:
    print("Solución encontrada:", solution.state)
else:
    print("No se encontró solución.")

********************************************HEURISTICA MANHATTAN*******************************************************
N es:  4
entrooo

node.state es:   (('S', (1, 1)), ('H', (1, 3)), ('L', (2, 2)), ('V', (3, 0)))
2
2
0
3
[(1, (1, 0)), (1, (1, 2)), (1, (1, 4)), (2, (1, 2)), (2, (3, 2)), (2, (2, 1)), (2, (2, 3)), (2, (3, 1)), (2, (3, 3)), (3, (0, 0)), (3, (1, 0)), (3, (2, 0)), (3, (4, 0))]
N es:  4
entrooo

node.state es:   (('S', (1, 1)), ('H', (1, 0)), ('L', (2, 2)), ('V', (3, 0)))
2
3
0
3
N es:  4
entrooo

node.state es:   (('S', (1, 1)), ('H', (1, 2)), ('L', (2, 2)), ('V', (3, 0)))
2
1
0
3
N es:  4
entrooo

node.state es:   (('S', (1, 1)), ('H', (1, 4)), ('L', (2, 2)), ('V', (3, 0)))
2
3
0
3
N es:  4
entrooo

node.state es:   (('S', (1, 1)), ('H', (1, 3)), ('L', (1, 2)), ('V', (3, 0)))
2
2
1
3
N es:  4
entrooo

node.state es:   (('S', (1, 1)), ('H', (1, 3)), ('L', (3, 2)), ('V', (3, 0)))
2
2
1
3
N es:  4
entrooo

node.state es:   (('S', (1, 1)), ('H', (1, 3)), ('L', (2, 1)), ('V',